# Initial Normalization of Images

In [1]:
from PIL import Image, ImageEnhance
import os
from tqdm import tqdm
import cv2

In [2]:
def normalize_images(dataset_dir, dir, new_dir, img):
    image = Image.open(os.path.join(dataset_dir, dir,img))
    new_image = image.resize((256, 256))
    new_image.save(os.path.join(dataset_dir, new_dir, img))

Resize Landscape images to 64x64

In [6]:
for i in tqdm(os.listdir('../dataset/landscape_dat/landscapes')):
    normalize_images('../dataset/landscape_dat', 'landscapes', 'landscape_256x256', i)

100%|██████████| 4319/4319 [02:26<00:00, 29.52it/s]


In [7]:
def color_to_bw(dataset_dir, dir, new_dir, img):
    image = Image.open(os.path.join(dataset_dir, dir, img))
    image_new = image.convert('L')
    image_new.save(os.path.join(dataset_dir, new_dir, img))

In [8]:
for i in tqdm(os.listdir('../dataset/landscape_dat/landscape_256x256')):
    color_to_bw('../dataset/landscape_dat', 'landscape_256x256', 'landscape_256x256_bw', i)

100%|██████████| 4319/4319 [00:39<00:00, 109.05it/s]


# Run each Model

# Testing and Creating Dataset

In [5]:
import cv2
import csv
from skimage.color import rgb2lab
import numpy as np
from tqdm import tqdm

def mse(imageA, imageB):
    # Mean Squared Error calculation
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    return err

def total_mse_2_images(image1_path, image_2_path):
    # Load images
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image_2_path)

    # Convert images to LAB space
    image1_lab = rgb2lab(image1)
    image2_lab = rgb2lab(image2)

    # Split LAB images into L, A, B channels
    L1, A1, B1 = cv2.split(image1_lab)
    L2, A2, B2 = cv2.split(image2_lab)

    # Calculate MSE for each channel
    mse_A = mse(A1, A2)
    mse_B = mse(B1, B2)

    return mse_A, mse_B

original = os.listdir('../dataset/landscape_dat/landscape_64x64/')
original = ['../dataset/landscape_dat/landscape_64x64/' + org for org in original]
infer = os.listdir('ddcolor/results/')
models = ['ddcolor', 'larsson_2016', 'siggraph_2016', 'zhang_2016']


In [ ]:
with open('dataset_64x64.csv', 'w', newline='') as csvfile:
    fieldnames = ['datatype', 'image', 'model']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    
    for org, inf in zip(original, infer):
        
        mse_total_A = []
        mse_total_B = []
        
        for data in models:
            inf_complete = f'{data}/results/'+ inf
            try:
                mse_A, mse_B = total_mse_2_images(org, inf_complete)
            except:
                mse_A, mse_B = 1000, 1000
            mse_total_A.append(mse_A)
            mse_total_B.append(mse_B)
        
        mse_total_A = np.array(mse_total_A)
        mse_total_B = np.array(mse_total_B)
        mse_total = (mse_total_A + mse_total_B)/2

        model = models[np.argmin(mse_total)]
        
        print(inf,model)
        writer.writerow({'datatype':'landscape', 'image':inf, 'model':model})

In [10]:
# Testing single image

mse_total_A = []
mse_total_B = []

org = '../dataset/landscape_dat/landscape_256x256/00000000_(2).jpg'

print(org)

for model in models:
    inf_complete = f'{model}/results_256x256/00000000_(2).jpg'
    try:
        mse_A, mse_B = total_mse_2_images(org, inf_complete)
    except:
        mse_A, mse_B = 1000, 1000
    mse_total_A.append(mse_A)
    mse_total_B.append(mse_B)

print(mse_total_A)
print(mse_total_B)

models = ['ddcolor', 'larsson_2016', 'siggraph_2016', 'zhang_2016']

../dataset/landscape_dat/landscape_256x256/00000000_(2).jpg
[200.21639326536896, 286.00041946645996, 207.64039857537875, 345.0714454340192]
[779.5754207575126, 2335.450302120792, 640.8708465659748, 1652.4554764326892]
